In [1]:
import os
import pandas as pd
import numpy as np
import random
import math
from math import log
from decimal import Decimal

In [2]:
count = 31
name = '3b02'

#path = '/home/katarina/'
path = 'C:/Kate/Bioinformatics/Diplom/'

os.chdir(path+"Set45/")
f = '.xlsx'

# List of AA to choose from for random change:
AA = ['A','R','N','D','C','E','Q','G','H','I','L','K','M','F','P','S','T','W','Y','V']

# DFs to hold the protein seq and B/E pattern
startProt = pd.DataFrame()

# Load parameters in dataframe dfCalc:
file = 'Params_FullSet_new.xlsx'
# Load spreadsheet
xf = pd.ExcelFile(file)    
# Load a sheet into a DataFrames by name: dfParams
dfParams = xf.parse('KBES')    # to be used for energy calc
dfCalc = dfParams              # to be used in entropy calc

# Load protein sequence and B/E pattern:
protFile = name+f
# Load spreadsheet
xf = pd.ExcelFile(protFile)    
# Load a sheet into a DataFrames by name of the protein
dfProtein = xf.parse('data')
protLength = len(dfProtein)
protLength

195

In [3]:
def calcEnergy(protein, dfParams): 
    energy = 0
    for num, r in enumerate(protein.Residue):
        if protein.b0e1.iloc[num] == 0:
            kbe = dfParams.KBenergy_RB[dfParams["Residue"] == r].item()
            energy = energy + kbe
        elif protein.b0e1.iloc[num] == 1:
            kbe = dfParams.KBenergy_RE[dfParams["Residue"] == r].item()
            energy = energy + kbe
    return energy

In [4]:
def factorialLog(number):
    flog = 0
    for n in range(1, number + 1):
          flog = flog + math.log(n)
    return(flog)

In [5]:
def logPow(a, b):
    x = b*math.log(a)
    return x

In [6]:
def calcLogEntropy(protein, dfCalc):
    # Table of frequencies in the protein    
    dfN = pd.crosstab(index = protein["Residue"], columns = protein["b0e1"])

    # Add a column of the number (n) of any residue class (40 classes)
    R = dfCalc.Residue.values
    B = []
    E = []
    for r in R:
        if r in dfN.index:
            if 0 in dfN.columns:
                B.append(dfN.at[r, 0])
            else:
                B.append(0)
            if 1 in dfN.columns:
                E.append(dfN.at[r, 1])
            else:
                E.append(0)
        else:
            B.append(0)
            E.append(0)
    dfCalc['B'] = B
    dfCalc['E'] = E
    
    # Calculate the frequencies on power of n
    PBpowN = []
    PEpowN = []
    for i in range(len(dfCalc)):
        PBpowN.append(logPow(dfCalc.F_RB[i], B[i]))
        PEpowN.append(logPow(dfCalc.F_RE[i], E[i]))
    # Calculate the LOG of frequencies on power of n
    dfCalc['PBpowN'] = PBpowN
    dfCalc['PEpowN'] = PEpowN
    
    # Calculate the LOG of factorials of n
    Bfactorial = []
    Efactorial = []
    for b in B:
        Bfactorial.append(factorialLog(b))
    for e in E:
        Efactorial.append(factorialLog(e))
    dfCalc['Bfactorial'] = Bfactorial
    dfCalc['Efactorial'] = Efactorial
    
    # Calculate the SUM (because of the log!!)
    nb = 1
    for i in dfCalc.Bfactorial:
        nb = nb+i
    ne = 1
    for i in dfCalc.Efactorial:
        ne = ne+i
    
    pb = 1
    for i in dfCalc.PBpowN:
        pb = pb+i
    pe = 1
    for i in dfCalc.PEpowN:
        pe = pe+i

    # cast as Decimal because of large numbers
    #pb = Decimal(pb)
    #pe = Decimal(pe)
    #lf = Decimal(math.factorial(dfCalc.B.sum()+dfCalc.E.sum()))  # const!!
    lf = factorialLog(dfCalc.B.sum()+dfCalc.E.sum())  # const!!
    #nb = Decimal(nb)
    #ne = Decimal(ne)
    
    #Calculate entropy
    entropy = (pb+pe+lf)-(nb+ne)
    
    return entropy

In [7]:
def calcEntropy(protein, dfCalc):
    # Table of frequencies in the protein    
    dfN = pd.crosstab(index = protein["Residue"], columns = protein["b0e1"])

    # Add a column of the number (n) of any residue class (40 classes)
    R = dfCalc.Residue.values
    B = []
    E = []
    for r in R:
        if r in dfN.index:
            if 0 in dfN.columns:
                B.append(dfN.at[r, 0])
            else:
                B.append(0)
            if 1 in dfN.columns:
                E.append(dfN.at[r, 1])
            else:
                E.append(0)
        else:
            B.append(0)
            E.append(0)
    dfCalc['B'] = B
    dfCalc['E'] = E
    
    # Calculate the frequencies on power of n
    dfCalc['PBpowN'] = dfCalc.F_RB.pow(B)
    dfCalc['PEpowN'] = dfCalc.F_RE.pow(E)
    
    # Calculate the factorials of n
    Bfactorial = []
    Efactorial = []
    for b in B:
        Bfactorial.append(Decimal(math.factorial(b)))
    for e in E:
        Efactorial.append(Decimal(math.factorial(e)))
    dfCalc['Bfactorial'] = Bfactorial
    dfCalc['Efactorial'] = Efactorial
    
    # Calculate the products
    nb = 1
    for i in dfCalc.Bfactorial:
        nb = nb*i
    ne = 1
    for i in dfCalc.Efactorial:
        ne = Decimal(ne*i)
    
    pb = 1
    for i in dfCalc.PBpowN:
        pb = Decimal(pb*Decimal(i))
    pe = 1
    for i in dfCalc.PEpowN:
        pe = Decimal(pe*Decimal(i))

    # cast as Decimal because of large numbers
    pb = Decimal(pb)
    pe = Decimal(pe)
    lf = Decimal(math.factorial(dfCalc.B.sum()+dfCalc.E.sum()))  # const!!
    nb = Decimal(nb)
    ne = Decimal(ne)
    
    #Calculate entropy
    entropy = (pb*pe*lf)/(nb*ne)
    
    return entropy

In [8]:
#Generate the starting protein:
# Add the columns to the starting dataframe
startProt['Residue'] = dfProtein.Residue
startProt['b0e1'] = dfProtein.b0e1

# Energy of the starting protein
energy = calcEnergy(startProt, dfParams)
energyStart = energy
eStartR = energyStart/protLength  # per residue

# Entropy of the starting protein 
entropy = calcEntropy(startProt, dfCalc)
entropyStart = entropy
entropyStartlog = calcLogEntropy(startProt, dfCalc)
sStartR = entropyStartlog/protLength   # per residue

# Entalpy of the starting protein 
entalpy = energyStart - entropyStartlog
entalpyStart = float(entalpy)
hStartR = entalpyStart/protLength   # per residue

In [9]:
repeats = 10
for n in range(repeats):

    # DFs to hold the protein seq and B/E pattern

    eprot = pd.DataFrame()   #for energy optimization
    sprot = pd.DataFrame()   #for entropy optimization
    hprot = pd.DataFrame()   #for entalpy optimization
    
    neprot = pd.DataFrame()   #for energy optimization
    nsprot = pd.DataFrame()   #for entropy optimization
    nhprot = pd.DataFrame()   #for entalpy optimization

    # DataFrames that will hold the optimized objects
    eprot['Residue'] = startProt.Residue
    eprot['b0e1'] = startProt.b0e1

    sprot['Residue'] = startProt.Residue
    sprot['b0e1'] = startProt.b0e1
    
    hprot['Residue'] = startProt.Residue
    hprot['b0e1'] = startProt.b0e1
    
    # DataFrames that will hold the objects with change on every cycle
    neprot['Residue'] = startProt.Residue
    neprot['b0e1'] = startProt.b0e1

    nsprot['Residue'] = startProt.Residue
    nsprot['b0e1'] = startProt.b0e1
    
    nhprot['Residue'] = startProt.Residue
    nhprot['b0e1'] = startProt.b0e1
    
    
    # Number of cylces
    cycles = protLength*15

    for i in range(cycles):
       
        # Generate random index and random AA
        rindex = random.randint(0, protLength-1)
                   
                
        for raa in AA: 
            # Make deep copies - eprot and sprot won't change when neprot and nsprot change
            neprot = eprot.copy()
            nsprot = sprot.copy()
            nhprot = hprot.copy()
            
            # Before the change:
                                                   
            # For entropy (S):
            
            # Calc entropy of nsprot:
    
            # Table of frequencies in the protein    
            dfN = pd.crosstab(index = nsprot["Residue"], columns = nsprot["b0e1"])
    
            # Add a column of the number (n) of any residue class (40 classes)
            R = dfCalc.Residue.values
            B = []
            E = []
            for r in R:
                if r in dfN.index:
                    if 0 in dfN.columns:
                        B.append(dfN.at[r, 0])
                    else:
                        B.append(0)
                    if 1 in dfN.columns:
                        E.append(dfN.at[r, 1])
                    else:
                        E.append(0)
                else:
                    B.append(0)
                    E.append(0)
            dfCalc['B'] = B
            dfCalc['E'] = E
            
            
            oldaa = nsprot.Residue.iloc[rindex]
    
            # If the residue that will change is buried:
            if nsprot.b0e1.iloc[rindex] == 0:
                Xold = float(dfCalc.B[dfCalc["Residue"] == oldaa])
                FXold = float(dfCalc.F_RB[dfCalc["Residue"] == oldaa]) 
                Yold = float(dfCalc.B[dfCalc["Residue"] == raa])
                FYold = float(dfCalc.F_RB[dfCalc["Residue"] == raa])
            # If the residue that will change is exposed:
            elif nsprot.b0e1.iloc[rindex] == 1:   
                Xold = float(dfCalc.E[dfCalc["Residue"] == oldaa])
                FXold = float(dfCalc.F_RE[dfCalc["Residue"] == oldaa]) 
                Yold = float(dfCalc.E[dfCalc["Residue"] == raa])
                FYold = float(dfCalc.F_RE[dfCalc["Residue"] == raa])
            
            # Make change to the new DF - nsprot only if this condition is true
            # Residue X is changed to residue Y
            if FYold*Xold > FXold*(Yold+1):
                nsprot.Residue.at[rindex] = raa
                sprot = nsprot.copy()  
                
                
            # For energy (E):
            
            # Before the change
            xaa = neprot.Residue.iloc[rindex]
            if neprot.b0e1.iloc[rindex] == 0:
                kbex = dfParams.KBenergy_RB[dfParams["Residue"] == xaa].item()
            elif neprot.b0e1.iloc[rindex] == 1:
                kbex = dfParams.KBenergy_RE[dfParams["Residue"] == xaa].item()
           
            # Make change to the new DF - neprot 
            # Residue X is changed to residue Y
            neprot.Residue.at[rindex] = raa
            
             # Calc new energy of neprot:    
            if neprot.b0e1.iloc[rindex] == 0:
                kbey = dfParams.KBenergy_RB[dfParams["Residue"] == raa].item()
            elif neprot.b0e1.iloc[rindex] == 1:
                kbey = dfParams.KBenergy_RE[dfParams["Residue"] == raa].item()
              
            # Optimization
            # If new energy is lower - change is accepted
            if kbey < kbex:
                eprot = neprot.copy()
                
            
            # For Entalpy (H):
            
            # Calc entropy of nhprot:    
            # Table of frequencies in the protein    
            dfN = pd.crosstab(index = nhprot["Residue"], columns = nhprot["b0e1"])
    
            # Add a column of the number (n) of any residue class (40 classes)
            R = dfCalc.Residue.values
            B = []
            E = []
            for r in R:
                if r in dfN.index:
                    if 0 in dfN.columns:
                        B.append(dfN.at[r, 0])
                    else:
                        B.append(0)
                    if 1 in dfN.columns:
                        E.append(dfN.at[r, 1])
                    else:
                        E.append(0)
                else:
                    B.append(0)
                    E.append(0)
            dfCalc['B'] = B
            dfCalc['E'] = E
            
            
            oldaa = nhprot.Residue.iloc[rindex]
    
            # If the residue that will change is buried:
            if nhprot.b0e1.iloc[rindex] == 0:
                Xold = float(dfCalc.B[dfCalc["Residue"] == oldaa])
                FXold = float(dfCalc.F_RB[dfCalc["Residue"] == oldaa]) 
                Yold = float(dfCalc.B[dfCalc["Residue"] == raa])
                FYold = float(dfCalc.F_RB[dfCalc["Residue"] == raa])
            # If the residue that will change is exposed:
            elif nhprot.b0e1.iloc[rindex] == 1:   
                Xold = float(dfCalc.E[dfCalc["Residue"] == oldaa])
                FXold = float(dfCalc.F_RE[dfCalc["Residue"] == oldaa]) 
                Yold = float(dfCalc.E[dfCalc["Residue"] == raa])
                FYold = float(dfCalc.F_RE[dfCalc["Residue"] == raa])
            
                                    
            # Check energy            
            # Before the change
            xaa = nhprot.Residue.iloc[rindex]
            if nhprot.b0e1.iloc[rindex] == 0:
                kbex = dfParams.KBenergy_RB[dfParams["Residue"] == xaa].item()
            elif nhprot.b0e1.iloc[rindex] == 1:
                kbex = dfParams.KBenergy_RE[dfParams["Residue"] == xaa].item()
           
            # Make change to the new DF - neprot 
            # Residue X is changed to residue Y
            nhprot.Residue.at[rindex] = raa
            
             # Calc new energy of neprot:    
            if nhprot.b0e1.iloc[rindex] == 0:
                kbey = dfParams.KBenergy_RB[dfParams["Residue"] == raa].item()
            elif nhprot.b0e1.iloc[rindex] == 1:
                kbey = dfParams.KBenergy_RE[dfParams["Residue"] == raa].item()
            
            
            # Calculate Entropy condition:
            increaseS = math.log((FYold*Xold)/(FXold*(Yold+1)))            
            # Calculate Energy condition:
            decreaseE = kbex - kbey            
            # Optimization
            # If the sum of both conditions is better - change is accepted
            if increaseS + decreaseE > 0:
                hprot = nhprot.copy()
                        
    
    # Extract the seuences
    nativeProt = ''
    for r in startProt.Residue:
        nativeProt = nativeProt + r
    
    optEnergy = ''
    for r in eprot.Residue:
        optEnergy = optEnergy + r

    optEntropy = ''
    for r in sprot.Residue:
        optEntropy = optEntropy + r
    
    optEntalpy = ''
    for r in hprot.Residue:
        optEntalpy = optEntalpy + r
    
    
    # Calculate the energy, entropy and entalpy of the protein with optimal energy
    # Energy
    energyOfEprot = calcEnergy(eprot, dfParams)
    energyER = energyOfEprot/protLength         # per residue
    # Entropy
    entropyOfEprot = calcEntropy(eprot, dfCalc)  
    entropyERlog = calcLogEntropy(eprot, dfCalc)
    entropyER = entropyERlog/protLength         # per residue
    # Entalpy
    entalpyOfEprot = energyOfEprot - entropyERlog  
    entalpyER = float(entalpyOfEprot)/protLength         # per residue
    
    # Calculate the energy, entropy and entalpy of the protein with optimal entropy
    # Energy
    energyOfSprot = calcEnergy(sprot, dfParams)
    energySR = energyOfSprot/protLength         # per residue
    # Entropy
    entropyOfSprot = calcEntropy(sprot, dfCalc)
    entropySRlog = calcLogEntropy(sprot, dfCalc)
    entropySR = entropySRlog/protLength         # per residue
    # Entalpy
    entalpyOfSprot = energyOfSprot - entropySRlog  
    entalpySR = float(entalpyOfSprot)/protLength         # per residue
        
    # Calculate the energy, entropy and entapy of the protein with optimal entalpy
    energyOfHprot = calcEnergy(hprot, dfParams)
    energyHR = energyOfHprot/protLength         # per residue
    # Entropy
    entropyOfHprot = calcEntropy(hprot, dfCalc)  
    entropyHRlog = calcLogEntropy(hprot, dfCalc)
    entropyHR = entropyHRlog/protLength         # per residue
    # Entalpy
    entalpyOfHprot = energyOfHprot - entropyHRlog  
    entalpyHR = float(entalpyOfHprot)/protLength         # per residue
    
    
    
    # Create a DF for the results:
    dfResults = pd.DataFrame()
    labels = [('Start_Nat'), 'OptE', 'OptS', 'OptH']

    # Create lists for the results:
    EperR = []
    SperR = []
    HperR = []
    resultEnergy = []
    resultEntropyLog = []
    resultEntalpy = []
    resultEntropy =[]
    resultDecimalEntropy = []
    resultSequence = []

    # Add E per Residue to list for the results:
    EperR.append(eStartR)  # of starting prot
    EperR.append(energyER)  # of prot with opt energy
    EperR.append(energySR)  # of prot with opt entropy
    EperR.append(energyHR)  # of prot with opt entalpy
    
    # Add S per Residue to list for the results:
    SperR.append(sStartR)  # of starting prot
    SperR.append(entropyER)  # of prot with opt energy
    SperR.append(entropySR)  # of prot with opt entropy
    SperR.append(entropyHR)  # of prot with opt entalpy
    
    # Add S per Residue to list for the results:
    HperR.append(hStartR)  # of starting prot
    HperR.append(entalpyER)  # of prot with opt energy
    HperR.append(entalpySR)  # of prot with opt entropy
    HperR.append(entalpyHR)  # of prot with opt entalpy
    
    # Add energy to list for the results:
    resultEnergy.append(energyStart)  # of starting prot
    resultEnergy.append(energyOfEprot)  # of prot with opt energy
    resultEnergy.append(energyOfSprot)  # of prot with opt entropy
    resultEnergy.append(energyOfHprot)  # of prot with opt entalpy
    
    # Add entropy (log) to list:
    resultEntropyLog.append(float(entropyStartlog))  # of starting prot
    resultEntropyLog.append(float(entropyERlog))  # of prot with opt energy
    resultEntropyLog.append(float(entropySRlog))  # of prot with opt entropy
    resultEntropyLog.append(float(entropyHRlog))  # of prot with opt entalpy
    
    # Add entalpy to list:
    resultEntalpy.append(float(entalpyStart))  # of starting prot
    resultEntalpy.append(float(entalpyOfEprot))  # of prot with opt energy
    resultEntalpy.append(float(entalpyOfSprot))  # of prot with opt entropy
    resultEntalpy.append(float(entalpyOfHprot))  # of prot with opt entalpy

    # Add entropy to list:
    resultEntropy.append(float(entropyStart))  # of starting prot
    resultEntropy.append(float(entropyOfEprot))  # of prot with opt energy
    resultEntropy.append(float(entropyOfSprot))  # of prot with opt entropy
    resultEntropy.append(float(entropyOfHprot))  # of prot with opt entalpy

    # Add decimal entropy to list:
    resultDecimalEntropy.append(entropyStart)  # of starting prot
    resultDecimalEntropy.append(entropyOfEprot)  # of prot with opt energy
    resultDecimalEntropy.append(entropyOfSprot)  # of prot with opt entropy
    resultDecimalEntropy.append(entropyOfHprot)  # of prot with opt entalpy

    # Add sequences to a list
    resultSequence.append(nativeProt)
    resultSequence.append(optEnergy)
    resultSequence.append(optEntropy)
    resultSequence.append(optEntalpy)

    dfResults['Label'] = labels
    dfResults['EperR'] = EperR
    dfResults['SperR'] = SperR
    dfResults['HperR'] = HperR
    dfResults['Energy'] = resultEnergy
    dfResults['EntropyLog'] = resultEntropyLog
    dfResults['Entalpy'] = resultEntalpy
    dfResults['Entropy'] = resultEntropy
    dfResults['Decimal Entropy'] = resultDecimalEntropy
    dfResults['Sequence'] = resultSequence
    
        
    # Save in file 
    os.chdir(path+"A3_AA/")
    
    writer = pd.ExcelWriter(('A3_AA'+str(count)+'_'+name+'_Native'+str(n)+'.xlsx'), engine='xlsxwriter')

    # Write the DataFrames to file sheets  
    dfResults.to_excel(writer, 'Results')
    startProt.to_excel(writer, 'NativeProt')
    eprot.to_excel(writer, 'OptEnergyProt')
    sprot.to_excel(writer, 'OptEntropyProt')
    hprot.to_excel(writer, 'OptEntalpyProt')

    # Save the result 
    writer.save()